In [2]:
import os 
import sys
from pathlib import Path
from tqdm import tqdm
from PIL import Image

current_file_dir = os.getcwd()

root_dir = os.path.dirname(current_file_dir)

sys.path.append(root_dir)
dataset_dir = os.path.join(root_dir, 'data')


processed_dir = os.path.join(dataset_dir, 'processed')
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)




In [3]:
from src.utils.preprocess import create_pointcloud_image, create_morphological_polygon


In [4]:
from nuscenes import NuScenes

In [5]:
## GLOBAL VARIABLES
filtered_categories = [24] # flat.drivable_surface

target_image_size = (398, 224)
dot_size = 5

pointsensor_channel = 'LIDAR_TOP'
camera_channel = 'CAM_FRONT'

In [6]:
def extract_lidarseg_data(nusc, save_dir, filtered_categories, target_image_size, dot_size, pointsensor_channel, camera_channel):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    outputs_type = ["cam", "masked", "pc"]
    for output_type in outputs_type:
        output_dir_type = os.path.join(save_dir, output_type)
        if not os.path.exists(output_dir_type):
            os.makedirs(output_dir_type)

    
    counter = 0
    for sample in tqdm(nusc.sample, desc="Processing samples"):
        token = sample['token']
        try:
            points, coloring, pil_image = nusc.explorer.map_pointcloud_to_image(
                pointsensor_token=sample['data'][pointsensor_channel],
                camera_token=sample['data'][camera_channel],
                filter_lidarseg_labels=filtered_categories,
                render_intensity=False,
                show_panoptic=False,
                show_lidarseg=True
            )
        except Exception as e:
            print(f"Error processing sample {token}: {e}")
            continue

        if points is None or coloring is None or pil_image is None:
            print(f"Skipping sample {token} due to missing data.")
            continue

        resized_image = pil_image.resize(target_image_size, Image.ANTIALIAS)
        point_cloud_image = create_pointcloud_image(
                                        points=points,
                                        coloring=coloring,
                                        image_size=pil_image.size,
                                        dot_size=dot_size,
                                        ignore_white=False
                                    )   
    point_cloud_image = point_cloud_image.resize(target_image_size, Image.ANTIALIAS)

    masked_image = create_morphological_polygon(
                                    points=points,
                                    coloring=coloring,
                                    image_size=pil_image.size,
                                    radius=5,
                                    )
    masked_image = masked_image.resize(target_image_size, Image.ANTIALIAS)

    counter += 1
    if counter % 1000 == 0:
        print(f"Processed {counter} samples.")
    resized_image.save(os.path.join(output_dir, f"cam/{token}.png"))
    masked_image.save(os.path.join(output_dir, f"masked/{token}.png"))
    point_cloud_image.save(os.path.join(output_dir, f"pc/{token}.png"))
    print(f"All samples processed and images saved in {output_dir}.")
    print(f"Processed images saved in {output_dir}.")
    
    

In [ ]:

## extract lidarseg data trainval
nusc = NuScenes(version='v1.0-trainval', dataroot=dataset_dir +"/sets/nuscenes", verbose=True)

output_dir = os.path.join(processed_dir, 'lidarseg_images/trainval')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


extract_lidarseg_data(
    nusc=nusc,
    save_dir=output_dir,
    filtered_categories=filtered_categories,
    target_image_size=target_image_size,
    dot_size=dot_size,
    pointsensor_channel=pointsensor_channel,
    camera_channel=camera_channel
)

In [7]:
nusc_test = NuScenes(version='v1.0-test', dataroot=dataset_dir +"/sets/nuscenes", verbose=True)
output_dir_test = os.path.join(processed_dir, 'lidarseg_images/test')
if not os.path.exists(output_dir_test):
    os.makedirs(output_dir_test)

## extract lidarseg data test

extract_lidarseg_data(
    nusc=nusc_test,
    save_dir=output_dir_test,
    filtered_categories=filtered_categories,
    target_image_size=target_image_size,
    dot_size=dot_size,
    pointsensor_channel=pointsensor_channel,
    camera_channel=camera_channel
)

Loading NuScenes tables for version v1.0-test...


FileNotFoundError: [Errno 2] No such file or directory: '/home/yasin/Lfstorage/Projects/autonomous-vehicle-drivable-area-segmentation/data/sets/nuscenes/v1.0-test/attribute.json'